In [1]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="udacity-project") - had to replace this line with Workspace.from_config()
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="quick-starts-ws-138316")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R5G892S8T to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-138316
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-138316


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = 'lesson2-cluster'
try:
    cluster = ComputeTarget(workspace = ws, name = cluster_name)
    print("Cluster already exists, start using it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", max_nodes = 4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output = True)

Cluster already exists, start using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [17]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice 
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
    "--C": uniform(0.0, 1.0),
    "--max_iter": choice(10, 25, 50, 75, 100)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1, delay_evaluation = 5)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# pip_packages = []
# est = SKLearn(
# 	source_directory='.',
# 	compute_target=cluster,
# 	entry_script = "train.py",
# 	pip_packages=pip_packages
# )

# since SKLearn estimator is deprecated, use ScriptRunConfig
from azureml.core import ScriptRunConfig
from azureml.core.environment import Environment

env = Environment.get(workspace = ws, name = "AzureML-Tutorial")

src = ScriptRunConfig(
    source_directory='.',
    script = "train.py",
    compute_target = cluster,
    environment = env
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
	run_config = src,
	hyperparameter_sampling=ps,
	policy=policy,
    primary_metric_name="AUC_weighted",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 50,
    max_duration_minutes = 60
)


In [18]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_f9281a2f-84e3-4f05-ba89-fa7be74e0c75
Web View: https://ml.azure.com/experiments/quick-starts-ws-138316/runs/HD_f9281a2f-84e3-4f05-ba89-fa7be74e0c75?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-138316/workspaces/quick-starts-ws-138316

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-10T04:35:36.385567][API][INFO]Experiment created<END>\n"<START>[2021-02-10T04:35:38.2103457Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']
print("\nBest run metrics:", best_run_metrics)
print("\nParameter values:", parameter_values)

# best_model = GradientBoostingRegressor(max_depth = 2, learning_rate = 1).fit(x_train, y_train)
best_model_filename = "hyperdrive_best_model.pkl" 

output_folder='./outputs'
os.makedirs(output_folder, exist_ok=True)
joblib.dump(value=best_model, filename=os.path.join(output_folder, best_model_filename))

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="AUC_weighted",
    training_data=,
    label_column_name="y",
    n_cross_validations=3)

In [2]:
# Submit your automl run

experiment_name = 'project-lesson2-automl'
experiment = Experiment(ws, experiment_name)
run = experiment.submit(automl_config)

RunDetails(run).show()

run.wait_for_completion(show_output=True)


In [ ]:
# Retrieve and save your best automl model.

best_automl_run, best_automl_model = run.get_output()
run_details = best_automl_run.get_details()
print("RUN_DETAILS", run_details)

print("PROPERTIES", run_details['properties'])

model_details = {
    'RunID': [run_details['runId']],
    'Iteration': [run_details['properties']['iteration']],
    'Primary metric': [run_details['properties']['primary_metric']],
    'Score': [run_details['properties']['score']],
    'Algorithm': [best_model.steps[1][0]],
    'Hyperparameters': [best_model.steps[1][1]]
}

model_file_name = 'automl_best_model.pkl'
joblib.dump(value=best_automl_model, filename=os.path.join(output_folder, model_file_name))


In [ ]:
# delete_compute(cluster)